pdf


In [5]:
s3_url = "parent/child/testing/file.pdf"  

import os
def get_file_name(s3_url:str) ->str:
    return os.path.basename(s3_url).split('/')[-1]
   
def change_file_ext(file_name:str, extension:str) ->str:
    file_path = file_name.split(".")
    file_path[-1]=extension
    
    return ".".join(file_path)

file_name = get_file_name(s3_url) 
print(file_name)
updated_file = change_file_ext(file_name, "html")
print(updated_file)

file.pdf
file.html
